<a href="https://colab.research.google.com/github/joaomigueld3/natural-language-processing/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Gensim é conhecido por sua implementação eficiente do modelo Word2Vec.
# Ele permite treinar modelos de representação de palavras usando grandes
# conjuntos de dados textuais.
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [ ]:
#Atualiza numpy e gensim sem cache (força download do mais recente)
#Reinicia automaticamente o ambiente do Colab (necessário para atualizar dependências)
!pip install --upgrade numpy gensim --no-cache-dir
import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.6 MB/s eta 0:00:00


### Repositório de Vetores Treinados

Os repositórios trazem vetores gerados a partir de um grande córpus do português do Brasil e português europeu, de fontes e gêneros variados.

In [ ]:
import numpy as np
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Corpus de texto
#corpus = [
#    "No meio do caminho tinha uma pedra",
#    "Tinha uma pedra no meio do caminho",
#    "Tinha uma pedra no meio do caminho",
#    "Tinha uma pedra",
#    "Nunca me esquecerei desse acontecimento",
#    "Na vida de minhas retinas tão fatigadas",
#    "Nunca me esquecerei",
#    "Que no meio do caminho tinha uma pedra",
#    "Tinha uma pedra no meio do caminho",
#    "No meio do caminho tinha uma pedra"
#]

corpus = [
    "A sustentabilidade é um tema central para o futuro do planeta.",
    "Empresas têm investido cada vez mais em energia renovável para reduzir o impacto ambiental.",
    "O consumo consciente pode mudar o mundo.",
    "Governos ao redor do mundo discutem políticas para mitigar os efeitos das mudanças climáticas.",
    "A preservação da biodiversidade é fundamental para o equilíbrio do ecossistema.",
    "A reciclagem é uma das formas mais simples de ajudar o meio ambiente.",
    "Projetos de energia solar têm crescido exponencialmente nos últimos anos.",
    "É urgente a transição para uma economia de baixo carbono.",
    "As ações para preservar os oceanos são essenciais para combater a poluição por plástico.",
    "A educação ambiental é a chave para conscientizar a sociedade sobre a importância da sustentabilidade."
]

# Tokenização das sentenças
tokenized_corpus = [word_tokenize(sentence.lower()) for sentence in corpus]

# Treinamento do modelo Word2Vec
# sentences: A lista de sentenças tokenizadas.
# vector_size: A dimensão dos vetores de palavra.
# window: O número máximo de palavras entre a palavra-alvo e as palavras ao seu redor.
# sg=0 CBOW, sg=1 SKIP-GRAM
# min_count: Ignora todas as palavras com frequência total inferior a esse valor.
# workers: O número de threads de treinamento paralelo para acelerar o treinamento.
model = Word2Vec(sentences=tokenized_corpus, vector_size=3, window=5, sg=0, min_count=1, workers=4)

# Salvando o modelo
model.save("word2vec_model")

#Treinando o modelo
# Treinamento do modelo
model.train(tokenized_corpus, total_examples=len(tokenized_corpus), epochs=10)




In [ ]:
# Exemplo de como obter o vetor de uma palavra
sustentabilidade = model.wv['sustentabilidade']
mundo = model.wv['mundo']
energia = model.wv['energia']

print("Vetor da palavra 'sustentabilidade':", sustentabilidade)

def dist_euclidiana(u,v):
  return np.linalg.norm(u - v)

#Imprimindo os caminhos
d1 = dist_euclidiana(sustentabilidade,mundo)
d2 = dist_euclidiana(mundo,sustentabilidade)
d3 = dist_euclidiana(sustentabilidade,energia)
d4 = dist_euclidiana(mundo,energia)

print("Distância entre sustentabilidade e mundo ", d1)
print("Distância entre mundo e sustentabilidade ", d2)
print("Distância entre sustentabilidade e energia ", d3)
print("Distância entre mundo e energia ", d4)

In [ ]:
def similaridade_cossenos(u,v):
  return np.dot(u,v)/(np.linalg.norm(u)*np.linalg.norm(v))

s1 = similaridade_cossenos(sustentabilidade,mundo)
s2 = similaridade_cossenos(mundo,sustentabilidade)
s3 = similaridade_cossenos(sustentabilidade,energia)
s4 = similaridade_cossenos(mundo,energia)

print("Distância entre sustentabilidade e mundo ", s1)
print("Distância entre mundo e sustentabilidade ", s2)
print("Distância entre sustentabilidade e energia ", s3)
print("Distância entre mundo e energia ", s4)


In [ ]:
#Analogia

# Exemplo
# x1= mulher, x2=homem, y1=rei, y2 = ?
# Mulher - Homem = y2 - Rei
# y2 = Mulher - Homem + Rei
# positives: Mulher e Rei
# Negative: Homem
def analogia(x1,x2,y1):
  y2 = model.wv.most_similar(positive = [x1, y1], negative = [x2])
  return y2

print(analogia(mundo, sustentabilidade, energia))

# Usando wv.most_similar
# No caso do CBOW, você obtém palavras similares à palavra-chave, e no caso do Skip-gram,
# você obtém palavras contextuais para a palavra-chave fornecida.
similar_words = model.wv.most_similar(energia, topn=15)
print(similar_words)

[('mundo', 0.9682457447052002), ('exponencialmente', 0.8894082307815552), ('crescido', 0.8879284858703613), ('anos', 0.8688837885856628), ('das', 0.8634596467018127), ('economia', 0.8627527356147766), ('sobre', 0.8446546196937561), ('consciente', 0.7998927235603333), ('últimos', 0.7663198709487915), ('as', 0.7554173469543457)]
[('energia', 0.9999999403953552), ('do', 0.9702626466751099), ('poluição', 0.9681754112243652), ('ambiente', 0.9654175639152527), ('tema', 0.9628732204437256), ('essenciais', 0.94303959608078), ('urgente', 0.8962976932525635), ('mudanças', 0.8893173336982727), ('reduzir', 0.8885645866394043), ('as', 0.8789161443710327), ('ao', 0.8566858172416687), ('consciente', 0.8508978486061096), ('sustentabilidade', 0.8504075407981873), ('mudar', 0.8313300013542175), ('efeitos', 0.7987024784088135)]
